<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/Digit%20Recognizer/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Library
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
path_train = '/content/drive/My Drive/Datasets/Digit Recognizer/train.csv'
path_test = '/content/drive/My Drive/Datasets/Digit Recognizer/test.csv'
#Read Data
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)
# Convert to numpy
train = np.asarray(train, np.float32)
test = np.asarray(test, np.float32)

X_train = train[:, 1:]
y_train = np.reshape(train[:, :1], (train.shape[0], ))
del train
del path_train
del path_test

In [0]:
# Training Parameters
learning_rate = 0.001
num_steps = 3000
batch_size = 128

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.25 # Dropout, probability to drop a unit


In [0]:
# Create Model
# Any results you write to the current directory are saved as output.
def conv_net(X, n_classes, dropout, reuse, is_training):
    with tf.variable_scope('Convnet', reuse=reuse):
        x=X['images']
        x=tf.reshape(x, shape=[-1, 28, 28, 1])
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        fc1 = tf.contrib.layers.flatten(conv2)
        fc1 = tf.layers.dense(fc1, 1024)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        out = tf.layers.dense(fc1, n_classes)
        
    return out
def model_fn(features, labels, mode):
    logits_train = conv_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = conv_net(features, num_classes, dropout, reuse=True, is_training=False)
    prediction = {
      "classes": tf.argmax(input=logits_test, axis=1),
      "probabilities": tf.nn.softmax(logits_test, name="softmax_tensor")
    }
    if(mode==tf.estimator.ModeKeys.PREDICT):
        return tf.estimator.EstimatorSpec(mode, predictions=prediction)
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    acc_op = tf.metrics.accuracy(labels=labels, predictions=prediction['classes'])
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=prediction,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})
    return estim_specs


In [6]:
model = tf.estimator.Estimator(model_fn)

input_fn_train = tf.estimator.inputs.numpy_input_fn(x={'images': X_train}, y=y_train, batch_size=batch_size, num_epochs=None, shuffle=True)

# Training Model
model.train(input_fn_train, steps=num_steps)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpsos5vsn8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2a76df0d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, u

In [7]:
# Test model 
input_fn_test = tf.estimator.inputs.numpy_input_fn(
    x={"images": test},
    num_epochs=1,
    shuffle=False)
predictions = model.predict(input_fn=input_fn_test)
cls = [p['classes'] for p in predictions]
cls_pred = np.array(cls, dtype='int').squeeze()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpsos5vsn8/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
# Convert to csv to submit to Kaggle Competitive
submit = pd.DataFrame()
submit['ImageId'] = range(1, test.shape[0]+1)
submit['Label'] = cls_pred
submit.to_csv("submit.csv", index=False)
